In [21]:
# Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn
from scipy import stats
from numpy import median
from sklearn import metrics
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingRegressor, BaggingRegressor
import lightgbm as lgb
import xgboost as xgb
import catboost as ctb
from mlxtend.regressor import StackingRegressor

filename = 'train.csv' 
df_train = pd.read_csv(filename)

# read test data
df_test = pd.read_csv('test.csv')

# rename some columns
df_train.rename(columns={'weathersit':'weather',
                     'mnth':'month',
                     'hr':'hour',
                     'yr':'year',
                     'hum': 'humidity',
                     'cnt':'count'},inplace=True)


# change some int values to categorical ones
df_train['season'] = df_train.season.astype('category')
df_train['year'] = df_train.year.astype('category')
df_train['month'] = df_train.month.astype('category')
#df_train['hour'] = df_train.hour.astype('category')
df_train['holiday'] = df_train.holiday.astype('category')
#df_train['weekday'] = df_train.weekday.astype('category')
df_train['workingday'] = df_train.workingday.astype('category')
df_train['weather'] = df_train.weather.astype('category')

df_train = df_train.drop(['atemp', 'windspeed', 'casual', 'registered'], axis=1)

df_train['count']=df_train['count'].apply(lambda x:np.log(x))

df_train['weather'] = df_train['weather'].replace(4,3) 
df_train = pd.get_dummies(df_train)
df_train = df_train.drop(df_train[['weather_4']], axis = 1)

X = df_train.drop(df_train[['count']], axis = 1)
y = df_train['count']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test

,hour,weekday,temp,humidity,season_1,season_2,season_3,season_4,year_0,year_1,...,month_10,month_11,month_12,holiday_0,holiday_1,workingday_0,workingday_1,weather_1,weather_2,weather_3
8522,1,1,0.62,0.88,0,0,1,0,0,1,...,0,0,0,1,0,0,1,1,0,0
4046,22,6,0.58,0.46,0,1,0,0,0,1,...,0,0,0,1,0,1,0,1,0,0
8297,21,4,0.50,0.36,0,1,0,0,1,0,...,0,0,0,1,0,0,1,1,0,0
7118,5,6,0.58,0.94,0,0,1,0,1,0,...,0,0,0,1,0,1,0,1,0,0
10258,10,3,0.52,0.88,0,0,0,1,1,0,...,1,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9898,19,4,0.34,0.46,0,0,0,1,1,0,...,0,0,1,1,0,0,1,1,0,0
11335,12,6,0.70,0.32,0,1,0,0,1,0,...,0,0,0,1,0,1,0,1,0,0
8764,3,3,0.64,0.89,0,0,1,0,0,1,...,0,0,0,1,0,0,1,1,0,0
10606,0,1,0.48,0.82,1,0,0,0,0,1,...,0,0,0,1,0,0,1,1,0,0


### LightGBM

In [22]:
lgb_train = lgb.Dataset(X, y)
#lgb_train = lgb.Dataset(X_train, y_train)
#lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {'bagging_fraction': 0.21, 'feature_fraction': 0.55, 'learning_rate': 0.118, 'n_estimators': 590, 'objective': 'tweedie', 'tree_leaner': 'feature', 'tweedie_variance_power': 1.59}

# train
lgbm = lgb.train(params, lgb_train)

C:\Users\user\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [10]:
y_pred = lgbm.predict(X_test)

y_pred_exp = np.exp(y_pred)
y_test_exp = np.exp(y_test)
for i, y in enumerate(y_pred_exp):
    if y_pred_exp[i] < 0:
        y_pred_exp[i] = 0
        
print('RMSE:', np.sqrt(mean_squared_error(y_test_exp, y_pred_exp)))
print('RMSLE:', np.sqrt(mean_squared_log_error(y_test_exp, y_pred_exp)))
print('R2:', r2_score(y_test_exp, y_pred_exp))

RMSE: 46.2989402286831
RMSLE: 0.3107042727772725
R2: 0.9319654322372721


### XGBoost

In [23]:
xg_reg = xgb.XGBRegressor(objective = 'reg:tweedie', colsample_bytree = 0.75, min_child_weight = 4, learning_rate = 0.117, max_depth = 5,
                 n_estimators = 690, tweedie_variance_power = 1.5, n_jobs = -1)

#xg_reg.fit(X_train,y_train)
xg_reg.fit(X,y)

C:\Users\user\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\user\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.75, gamma=0,
             importance_type='gain', learning_rate=0.117, max_delta_step=0,
             max_depth=5, min_child_weight=4, missing=None, n_estimators=690,
             n_jobs=-1, nthread=None, objective='reg:tweedie', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, tweedie_variance_power=1.5, verbosity=1)

In [12]:
y_pred = xg_reg.predict(X_test)

y_pred_exp = np.exp(y_pred)
y_test_exp = np.exp(y_test)
for i, y in enumerate(y_pred_exp):
    if y_pred_exp[i] < 0:
        y_pred_exp[i] = 0
        
print('RMSE:', np.sqrt(mean_squared_error(y_test_exp, y_pred_exp)))
print('RMSLE:', np.sqrt(mean_squared_log_error(y_test_exp, y_pred_exp)))
print('R2:', r2_score(y_test_exp, y_pred_exp))

RMSE: 45.35743728357495
RMSLE: 0.30844644694495865
R2: 0.9347043052781682


### CatBoost

In [24]:
cat = ctb.CatBoostRegressor(silent = True,
                        loss_function='RMSE',
                        learning_rate=0.118,
                        l2_leaf_reg = 0.7)


#cat.fit(X_train, y_train)
cat.fit(X, y)

In [14]:
y_pred = cat.predict(X_test)

y_pred_exp = np.exp(y_pred)
y_test_exp = np.exp(y_test)
for i, y in enumerate(y_pred_exp):
    if y_pred_exp[i] < 0:
        y_pred_exp[i] = 0
        
print('RMSE:', np.sqrt(mean_squared_error(y_test_exp, y_pred_exp)))
print('RMSLE:', np.sqrt(mean_squared_log_error(y_test_exp, y_pred_exp)))
print('R2:', r2_score(y_test_exp, y_pred_exp))

RMSE: 38.94490523765621
RMSLE: 0.2823253233805982
R2: 0.951861912459604


### Stacking Regressor

In [28]:
stregr = StackingRegressor(regressors=[xg_reg, cat], 
                           meta_regressor=cat)
#stregr.fit(X_train, y_train)
stregr.fit(X, y)

TypeError: object of type 'numpy.float64' has no len()

In [ ]:
y_pred = stregr.predict(X_test)

y_pred_exp = np.exp(y_pred)
y_test_exp = np.exp(y_test)
for i, y in enumerate(y_pred_exp):
    if y_pred_exp[i] < 0:
        y_pred_exp[i] = 0
        
print('RMSE:', np.sqrt(mean_squared_error(y_test_exp, y_pred_exp)))
print('RMSLE:', np.sqrt(mean_squared_log_error(y_test_exp, y_pred_exp)))
print('R2:', r2_score(y_test_exp, y_pred_exp))

### Kaggle

In [26]:
filename = 'test.csv' 
df_test = pd.read_csv(filename)

df_test.rename(columns={'weathersit':'weather',
                     'mnth':'month',
                     'hr':'hour',
                     'yr':'year',
                     'hum': 'humidity',
                     'cnt':'count'},inplace=True)

df_test['season'] = df_test.season.astype('category')
df_test['year'] = df_test.year.astype('category')
df_test['month'] = df_test.month.astype('category')
#df_test['hour'] = df_test.hour.astype('category')
df_test['holiday'] = df_test.holiday.astype('category')
#df_test['weekday'] = df_test.weekday.astype('category')
df_test['workingday'] = df_test.workingday.astype('category')
df_test['weather'] = df_test.weather.astype('category')

df_test = df_test.drop(['atemp', 'windspeed'], axis=1)
df_test = pd.get_dummies(df_test)

y_pred = cat.predict(df_test)
y_pred = np.exp(y_pred)

for i, y  in enumerate(y_pred):
    if y_pred[i] < 0:
        y_pred[i] = 0

submission = pd.DataFrame()
submission['Id'] = range(y_pred.shape[0])
submission['Predicted'] = y_pred.astype(int)
submission.to_csv("submission_stacked.csv", index=False)